In [22]:
import pennylane as qml
from pennylane import numpy as np

In [23]:
#Want theta=pi/2, so RY(pi/2) creates |x> state for 1st qubit,
#and rest RY, CNOT, RY is actually inverse CNOT
#Measure X@X, which will give sin(theta)
def bell_plus_minus(theta):
    qml.RY(theta, wires=0)
    qml.RY(np.pi, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(-np.pi, wires=0)
    return qml.expval(qml.PauliX(0) @ qml.PauliX(1))

In [24]:
#Specify different numbers of measurements
num_meas = [1, 10, 100, 1000]

In [25]:
make_device = lambda samples: qml.device('default.qubit', wires=2, shots=samples, analytic=False)
#Create different devices with corresponding numbers of measurements
sampling_devices = map(make_device, num_meas)

make_qnode = lambda device: qml.QNode(bell_plus_minus, device)
#Create qnodes with same circuit, but different sampling devices
qnodes = map(make_qnode, sampling_devices)

In [26]:
#This is higher order function, which
#given QNode returns same cost function, but evaluated via that QNode
#Minimum of cost corresponds to theta=pi/2
def make_cost(qnode):
    def cost(theta):
        return (qnode(theta) - 1)**2
    return cost

In [27]:
#Create different cost functions, corresponding to same circuit, 
#but different numbers of measurements
qnodes_costs = list(map(make_cost, qnodes))

In [28]:
#Optimize theta for given cost function(made of QNode)
def optimize(qnode_cost):
    opt = qml.GradientDescentOptimizer(stepsize=0.4)
    params = 0.01
    steps = 1000
    for i in range(steps):
        params = opt.step(qnode_cost, params)
    return params

In [29]:
#Find thetas for different numbers of measurements
thetas = list(map(optimize, qnodes_costs))

In [30]:
print(thetas)

[1.61, 1.5460000000000003, 1.5404000000000013, 1.5352128]
